In [ ]:
'''
This notebook contains the code to merge all male data across the years based on sections.
There are datasets for 2012, 2012-1.5, 2013 and 2014
The different sections that will be merged are as follows:
**2012**
1. Cover
2. Roaster
3. Section 1: Education (All men 18 and above)
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Economic Events/Shocks
10. Section 8: Community Participation and Social Network Membership

**2013**
1. Cover
2. Roaster
3. Section 1: Education: Males 19 years and older
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Health
10. Section 8: Political Participation and Governance

**2014**
1. Cover
2. Roaster
3. Section 1: Education: Males 19 years and older
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Economic Events/Shocks”
10. Section 8: Participation in Social Safety Net
11. Section 9: Siblings
12. Section 10: Transfers
13. Section 11: Health and Nutrition

'''

In [ ]:
'''
The following code block will merge all roaster files across the years.
* First, we are going to read the respective files and store them as data frames
* Next, we are going to define column mappings that I have already figured out via manual methods
* Once the mappings are done per the set rules, we will see the new roaster dataset across the years 1.
'''

In [32]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes
import numpy as np
import pandas as pd

# Store excel file locations to variables

agri_2012_5 = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\12. Section 2 Part 10 Income from Livestock\2012_5_s1p9_q6to17.xlsx"
agri_2014= r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\12. Section 2 Part 10 Income from Livestock\2014_s2p13_m.xlsx"
# Read excel files 
df_2012_5 = pd.read_excel(agri_2012_5)
df_2014 = pd.read_excel(agri_2014)


In [33]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df
df_2012_5.rename(columns={

    'Round': 'round',
    'Code': 's2p13_code',
    'ITEM_NAME': 's2p13_name',
    'S1P9Q6': 's2p13_q1',
    'S1P9Q7': 's2p13_q2',
    'S1P9Q8': 's2p13_q3_qty',
    'S1P9Q9': 's2p13_q3_unit',
    'S1P9Q10': 's2p13_q4',
    'S1P9Q11': 's2p13_q5',
    'S1P9Q12': 's2p13_q6',
    'S1P9Q16A': 's2p13_q7',
    'S1P9Q16B': 's2p13_q8',
    'S1P9Q13': 's2p13_q10',
    'S1P9Q14': 's2p13_q11',
    'S1P9Q15': 's2p13_q12',
    'S1P9Q17': 'Total_Val_S',
    'C_PROVINCE': 'P_ID',
    'C_DISTRICT': 'D_ID',
    'C_TEHSIL': 'T_ID',
    'C_UC': 'UC_ID',
    'C_MOUZA': 'M_ID'
}, inplace=True)



# df_2014 doesn't need renaming as it is the reference

In [34]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2012_5 = [
    "hid", "round", "s2p13_code", "s2p13_name", "s2p13_q1", "s2p13_q2",
    "s2p13_q3_qty", "s2p13_q3_unit", "s2p13_q4", "s2p13_q5", "s2p13_q6",
    "s2p13_q7", "s2p13_q8", None, "s2p13_q9", "s2p13_q10", "s2p13_q11",
    "s2p13_q12", "Total_Val_S", "P_ID", "D_ID", "T_ID", "UC_ID", "M_ID", "C_HH_NUM"
]

mapping_2014 = [
    "hid", "round", "s2p13_code", "s2p13_name", 
    "s2p13_q1", "s2p13_q2", "s2p13_q3_qty", "s2p13_q3_unit", 
    "s2p13_q4", "s2p13_q5", "s2p13_q6", "s2p13_q7", 
    "s2p13_q8", "s2p13_q9", "s2p13_q10", "s2p13_q11", 
    "s2p13_q12", None, None, None, None, None, None, None
]


In [35]:
# Create a list of all possible columns in the correct order
all_columns = []
for col in mapping_2012_5:
    if col and col not in all_columns:
        all_columns.append(col)

for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [36]:

# Function to standardize and merge DataFrames
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if col in df.columns:
                    print(f"Appending data for column {col}")
                    merged_data[ref_col].extend(df[col].tolist())
                else:
                    print(f"Column {col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df

In [37]:
# Usage with dataframes and mappings
dfs = [df_2012_5, df_2014]
mappings = [mapping_2012_5, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'round', 'hid', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID', 'C_HH_NUM', 's2p13_code', 's2p13_name', 's2p13_q1', 's2p13_q2', 's2p13_q3_qty', 's2p13_q3_unit', 's2p13_q4', 's2p13_q5', 's2p13_q6', 's2p13_q10', 's2p13_q11', 's2p13_q12', 's2p13_q7', 's2p13_q8', 'Total_Val_S']
Appending data for column hid
Appending data for column round
Appending data for column s2p13_code
Appending data for column s2p13_name
Appending data for column s2p13_q1
Appending data for column s2p13_q2
Appending data for column s2p13_q3_qty
Appending data for column s2p13_q3_unit
Appending data for column s2p13_q4
Appending data for column s2p13_q5
Appending data for column s2p13_q6
Appending data for column s2p13_q7
Appending data for column s2p13_q8
Column s2p13_q9 not found in DataFrame. Adding NaNs.
Appending data for column s2p13_q10
Appending data for column s2p13_q11
Appending data for column s2p13_q12
Appending data for column Total_Val_S
Appending data for colu

In [38]:
# Rename columns for the merged file (if needed)
rename_mapping={
    'hid': 'HID',
    'round': 'Survey_Round',
    's2p13_code': 'Item_code',
    's2p13_name': 'Item_name',
    's2p13_q1': 'Item_HH_P',
    's2p13_q2': 'Time_Prd',
    's2p13_q3_qty': 'Qty_P',
    's2p13_q3_unit': 'Unit_P',
    's2p13_q4': 'Val_T_P',
    's2p13_q5': 'Item_P_Mon',
    's2p13_q6': 'Item_S',
    's2p13_q7': 'Item_S_Qty',
    's2p13_q8': 'Item_S_Qty_Unit',
    's2p13_q9': 'Item_S_PPU',
    's2p13_q10': 'Item_S_pers',
    's2p13_q11': 'Item_S_Loc',
    's2p13_q12': 'Item_S_Dist'
}

merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_Section_2_part_10.csv', index=False)

